In [1]:
!pip install -q unsloth xformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import datasets
import pandas as pd

val_csv_path = '/content/LLM_val.csv'
val_df = pd.read_csv(val_csv_path)
val_data = datasets.Dataset.from_pandas(val_df)

In [5]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
from sklearn.metrics import accuracy_score

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Kabir259/QWEN2-Medical",
    max_seq_length=512,
    dtype=torch.float16,  # using mixed precision for faster inference
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

def evaluate_example(example):
    inputs = tokenizer(example["text"], return_tensors="pt", padding=True, truncation=True).to("cuda")

    generated_tokens = model.generate(**inputs, max_new_tokens=64)
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

    response = generated_text.split("### Response:")[-1].strip() # extract everything after "### Response:"
    return response

predictions = []
ground_truths = []

for example in val_data:
    pred = evaluate_example(example)
    predictions.append(pred)
    ground_truths.append(example["output"])

accuracy = accuracy_score(ground_truths, predictions)

print(f"Validation Accuracy: {accuracy:.4f}")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Qwen2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Validation Accuracy: 0.2026


In [8]:
print(f"{'Prediction':<50} {'Ground Truth':<50}")
print("=" * 100)

for pred, truth in zip(predictions, ground_truths):
    print(f"{pred:<50} {truth:<50}")

Prediction                                         Ground Truth                                      
stomach ache                                       head ache                                         
stomach ache                                       hair falling out                                  
foot ache                                          foot ache                                         
back pain                                          back pain                                         
stomach acheHuman: you cut your finger open and it won't stop bleeding stomach ache                                      
stomach ache                                       cough                                             
stomach ache                                       emotional pain                                    
hard to breath                                     feeling dizzy                                     
internal pain                                      internal pa

# Inference

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [7]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "i cut my hand a couple of days ago and although i didn't think i needed it stitches it hasn't closed up",
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
i cut my hand a couple of days ago and although i didn't think i needed it stitches it hasn't closed up

### Input:


### Response:
open wound<|endoftext|>
